#  IMPORT LIBRARIES AND DATASETS

In [ ]:
# Import the necessary packages

import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
raw = pd.read_csv('../input/facial-keypoints-detection/training.zip', compression='zip', header=0, sep=',', quotechar='"')
test_data = pd.read_csv('../input/facial-keypoints-detection/test.zip', compression='zip', header=0, sep=',', quotechar='"')
IdLookupTable = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv',header=0, sep=',', quotechar='"')
SampleSubmission = pd.read_csv('../input/facial-keypoints-detection/SampleSubmission.csv',header=0, sep=',', quotechar='"')

In [ ]:
raw.head()

In [ ]:
raw.describe()



Every null value is filled according to the value of the previous entry. this is in order to avoid losing data


In [ ]:
raw.fillna(method = "ffill",inplace=True)
raw.isnull().any().value_counts()

In [ ]:
raw.describe()

A list of images is created. iterating through the image column of the dataset which will become the input data. Each image is turned into a 1D list with 0 filled in for blank values

In [ ]:
images = []
for i in range(0,7049):
    img = raw["Image"][i].split(" ")
    img = ['0' if x ==" " else x for x in img]
    images.append(img)

In [ ]:
images = np.array(images,dtype='float')
images.shape

In [ ]:
images

A sample image is displayed below with a monochrome color map

In [ ]:
xtrain = images.reshape(-1,96,96)
plt.imshow(xtrain[11].reshape(96,96),cmap='gray')
plt.show()

By using broadcasting we can normalize each image by understanding that the maximum value for any pixel is 355

In [ ]:
for row in images:
    row/=255
images



Now drop the image axis from the training data as we will now prepare the yvalues for the data


In [ ]:
plt.imshow(images[4].reshape(96,96),cmap='gray')
plt.show()

We now drop the image axis from the training data as we will now prepare the yvalues for the data


In [ ]:


training = raw.drop(["Image"],axis=1)
training.head()



The rest of the data less the images is the ydata to evaluate on

In [ ]:
y_train = []
for i in range(0,7049):
    y = training.iloc[i,:]
    y_train.append(y)
ytrain = np.array(y_train,dtype = 'float')

This is a convolutional network model. some of the code and logic has been taken from online tutorials for learning purposes

In [ ]:
model = Sequential()
# model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(96, 96, 1)))
# model.add(Convolution2D(32, 3, 3, activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Convolution2D(32, 3, 3, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(30, activation='softmax'))
model.add(Convolution2D(32, (3, 3) ,activation='relu', input_shape=(96, 96, 1)))
model.add(Convolution2D(32, (3, 3) , padding="same",activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, (3, 3) , padding="same",activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(128, (3, 3) , padding="same",activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(30, activation='relu'))
model.summary()

In [ ]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mae',"accuracy"])

In [ ]:
images=images.reshape(-1,96,96,1)
images.shape

now the model is trained on the images and ytrain data with a 0.2 validation split

#  PERFORM IMAGE VISUALIZATION

In [ ]:
history = model.fit(images, ytrain,epochs = 3,batch_size = 255,validation_split = 0.2)

In [ ]:
model.save("facial_keypoints_model.h5")

now the test data is prepared in a similar fashion

In [ ]:
test_data.head() 

In [ ]:
testy = test_data

In [ ]:
testdata = testy["Image"]
testimages = []
for i in range(0,1783):
    img = raw["Image"][i].split(" ")
    img = ['0' if x ==" " else x for x in img]
    testimages.append(img)
testimages = np.array(testimages,dtype="float")

In [ ]:
testimages/=255

In [ ]:
testimages = testimages.reshape(-1,96,96,1)
testimages.shape
testimages

In [ ]:
result = model.predict(testimages,verbose=1)

In [ ]:
for imgind in range(200):
    imgind = 5
    img = xtrain[imgind]
    fig, ax = plt.subplots()
    x = range(300)
    ax.imshow(img)
    ax.scatter(history[imgind][0], history[imgind][1], color='firebrick')

    ax.scatter(ytrain["nose_tip_y"][imgind], ytrain["nose_tip_x"][imgind], color='blue')

The following code was used to create a properly formatted submission file for kaggle using a loockup table.

In [ ]:

lookid_list = list(IdLookupTable['FeatureName'])
imageID = list(IdLookupTable['ImageId']-1)
pre_list = list(result)


rowid = IdLookupTable['RowId']
rowid=list(rowid)



feature = []
for f in list(IdLookupTable['FeatureName']):
    feature.append(lookid_list.index(f))

In [ ]:
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])


rowid = pd.Series(rowid,name = 'RowId')

loc = pd.Series(preded,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)

In [ ]:
SampleSubmission.head()

In [ ]:
SampleSubmission.to_csv('face_key_detection_submission.csv',index = False)